# Champions general statistics

Get the saved games data

In [1]:
import json, os
gamesRawData = []
for i in os.listdir("./games"):
    if i.endswith(".json"):
        with open("./games/"+i,"r") as f:
            gamesRawData.append(json.loads(f.read()))

Initialize the champion stats data structure.

In [2]:
import requests

response = requests.get('http://ddragon.leagueoflegends.com/cdn/8.19.1/data/en_US/champion.json')
champonRawData = json.loads(response.text)

championsStats = {}
for key,champion in champonRawData['data'].items():
    championsStats[int(champion['key'])] = {
                    "champion":champion['name'],
                    "games":0,
                    "gamesPlayed":0,
                    "wins":0,
                    "teamKills":0,
                    "kills":0,
                    "deaths":0,
                    "assists":0,
                    "bans":0
                }


Parse all the games to get statistics by champion

In [3]:
for game in gamesRawData:
    #Count the champion bans
    for t in game["teams"]:
        for b in t["bans"]:
            championsStats[b["championId"]]["bans"] += 1
            championsStats[b["championId"]]["games"]+=1
    
    #Count the kills of the team to later get the kill participation
    teamKills = {100:0,200:0}
    for p in game["participants"]:
        teamKills[p["teamId"]] += p["stats"]["kills"]
    
    #Count the wins and basic stats like kills, deaths and assits
    for p in game["participants"]:
        championsStats[p["championId"]]["games"]+=1
        championsStats[p["championId"]]["gamesPlayed"]+=1
        if p["stats"]["win"]:
            championsStats[p["championId"]]["wins"] += 1
        
        championsStats[p["championId"]]["teamKills"] += teamKills[p["teamId"]]
        championsStats[p["championId"]]["kills"] += p["stats"]["kills"]
        championsStats[p["championId"]]["deaths"] += p["stats"]["deaths"]
        championsStats[p["championId"]]["assists"] += p["stats"]["assists"]

We make a difference between games and gamesPlayed where games counts every game where the champion has been either played or banned.

Use pandas Dataframe to manipulate the data

In [4]:
import pandas as pd, numpy as np
df = pd.DataFrame(championsStats).T

#Get the winrate for each champion
#wr is the raw winrate
df["wr"] = 0
df.loc[df["gamesPlayed"]>0,"wr"] = df.loc[df["gamesPlayed"]>0]["wins"]/df.loc[df["gamesPlayed"]>0]["gamesPlayed"]
#winrate is the beautified winrate
df["winrate"] = (df["wr"] * 100).map('{:,.2f}'.format).apply(lambda s: str(s)+"%")

#Get the KDA
df["KDA"] = 0
df.loc[df["deaths"]>0,"KDA"] = ((df.loc[df["deaths"]>0]["kills"] + df.loc[df["deaths"]>0]["assists"]) / df.loc[df["deaths"]>0]["deaths"])
#Handling the case when KDA is infinite (deaths=0)
df.loc[df["deaths"]==0,"KDA"]=(df.loc[df["deaths"]==0]["kills"] + df.loc[df["deaths"]==0]["assists"])
#Beautify KDA
df["KDA"] = df["KDA"].map('{:,.2f}'.format)

#Get kill participation and beautify it
df["KP"] = 0
df.loc[df["teamKills"]>0,"KP"] = (((df.loc[df["teamKills"]>0]["kills"] + df.loc[df["teamKills"]>0]["assists"]) / df.loc[df["teamKills"]>0]["teamKills"]) * 100).map('{:,.2f}'.format).apply(lambda s: str(s)+"%")

Resulting Dataframe

In [5]:
df

,assists,bans,champion,deaths,games,gamesPlayed,kills,teamKills,wins,wr,winrate,KDA,KP
1,0,0,Annie,0,0,0,0,0,0,0,0.00%,0.00,0
2,125,29,Olaf,53,54,25,73,281,10,0.4,40.00%,3.74,70.46%
3,129,27,Galio,55,54,27,64,290,13,0.481481,48.15%,3.51,66.55%
4,0,0,Twisted Fate,0,0,0,0,0,0,0,0.00%,0.00,0
5,163,23,Xin Zhao,78,54,31,77,371,17,0.548387,54.84%,3.08,64.69%
6,190,68,Urgot,115,115,47,140,551,26,0.553191,55.32%,2.87,59.89%
7,128,49,LeBlanc,54,76,27,109,381,18,0.666667,66.67%,4.39,62.20%
8,0,0,Vladimir,0,0,0,0,0,0,0,0.00%,0.00,0
9,0,0,Fiddlesticks,0,0,0,0,0,0,0,0.00%,0.00,0
10,0,0,Kayle,0,0,0,0,0,0,0,0.00%,0.00,0


Exporting the data to csv

In [6]:
df.index.name='championId'
df.to_csv("champion.csv")

Some manipulation examples

In [7]:
(df[["champion","kills","deaths","assists","KDA","teamKills","KP","wins","gamesPlayed","games","bans","winrate","wr"]] #Selecting the columns to show and their order
    .sort_values(["wr","gamesPlayed"], ascending=False) #Sorting champions by winrate and games played
    .drop("wr", axis=1)) #Drop the "wr" column which is only here for sorting purpose, the "winrate" column giving the same information but beautified

,champion,kills,deaths,assists,KDA,teamKills,KP,wins,gamesPlayed,games,bans,winrate
championId,,,,,,,,,,,,
27,Singed,7,3,14,7.00,35,60.00%,2,2,4,2,100.00%
43,Karma,3,2,16,9.50,24,79.17%,2,2,3,1,100.00%
154,Zac,3,1,21,24.00,31,77.42%,2,2,4,2,100.00%
157,Yasuo,2,6,21,3.83,28,82.14%,2,2,7,5,100.00%
34,Anivia,3,0,12,15.00,18,83.33%,1,1,5,4,100.00%
120,Hecarim,5,2,3,4.00,13,61.54%,1,1,3,2,100.00%
122,Darius,1,1,5,6.00,11,54.55%,1,1,5,4,100.00%
240,Kled,2,1,7,9.00,20,45.00%,1,1,1,0,100.00%
421,Rek'Sai,10,3,8,6.00,27,66.67%,1,1,1,0,100.00%


In [8]:
#Getting percentage of champions either played or banned in Worlds 2018
df[df["games"] > 0].shape[0] / df.shape[0]

0.6524822695035462

In [9]:
(df[["champion","kills","deaths","assists","KDA","teamKills","KP","wins","gamesPlayed","games","bans","winrate","wr"]] #Selecting the columns to show and their order
    .sort_values(["games"], ascending=False) #Sorting champions by games (played and banned)
    .drop("wr", axis=1)) #Drop the "wr" column which is only here for sorting purpose, the "winrate" column giving the same information but beautified

,champion,kills,deaths,assists,KDA,teamKills,KP,wins,gamesPlayed,games,bans,winrate
championId,,,,,,,,,,,,
266,Aatrox,91,105,167,2.46,497,51.91%,20,42,118,76,47.62%
6,Urgot,140,115,190,2.87,551,59.89%,26,47,115,68,55.32%
12,Alistar,29,122,342,3.04,553,67.09%,28,50,113,63,56.00%
84,Akali,83,39,76,4.08,247,64.37%,11,19,95,76,57.89%
145,Kai'Sa,311,115,282,5.16,849,69.85%,39,69,94,25,56.52%
497,Rakan,41,106,383,4.00,548,77.37%,24,44,87,43,54.55%
39,Irelia,115,96,146,2.72,406,64.29%,19,30,87,57,63.33%
223,Tahm Kench,18,66,173,2.89,309,61.81%,13,29,80,51,44.83%
498,Xayah,167,83,212,4.57,549,69.03%,22,44,78,34,50.00%
